In [ ]:
### SET DRIVE PATH ###
drive = "/SETDRIVEPATH/data_lsd_trip/"
drive_path = drive+'data_paus/YYYYMMDD_POSSIBLE_SET_NAME/'

### SPECIFY MITK BINARY PATH ###
### probably need to build your own
mitk_bin_path = "/PATHTO/MitkPABeamformingTool"

In [ ]:
import nrrd
import os
import scipy.io as sio
import h5py
import numpy as np
import matplotlib.pyplot as plt
import glob

# meta data
no_ill_pos = 4
no_wavelengths = 16
no_laser_pulses_in_sequence = no_ill_pos*no_wavelengths

acqusitions = glob.glob(drive_path+"acquisition_*")
raw_filepath = '/rawdata.mat'

In [ ]:
for acqusition in acqusitions:
    with h5py.File(acqusition+raw_filepath, 'r') as f:
        raw_buffer = np.asarray(f['rawbuffer'])
    print("[loaded]", acqusition, "with shape", raw_buffer.shape)

    no_frames_per_seq = raw_buffer[0,:,0,0].size
    no_sequences = raw_buffer[:,0,0,0].size

    raw_pa_sorted = np.zeros([raw_buffer[0,0,:,0].size,
                          raw_buffer[0,0,0,:].size,
                          no_sequences,
                          no_wavelengths,
                          no_ill_pos])

    for i_seq in range(no_sequences):
        for i_wav in range(no_wavelengths):
            for i_ill in range(no_ill_pos):
                raw_pa_sorted[:32, :, i_seq, i_wav, i_ill] = raw_buffer[i_seq, i_ill+i_wav*no_ill_pos, 32:, :]
                raw_pa_sorted[32:, :, i_seq, i_wav, i_ill] = raw_buffer[i_seq, i_ill+i_wav*no_ill_pos, :32, :]

    del raw_buffer

    std_transducer_left_m_right = np.std(raw_pa_sorted[:5, 20:40, ...], axis=(1,0)) - np.std(raw_pa_sorted[59:, 20:40, ...], axis=(1,0))
    mean_transducer = np.mean(np.abs(raw_pa_sorted[:, 20:40, ...]), axis=(1,0))
    mean_transducer_delta = np.zeros_like(mean_transducer)

    for k in range(no_ill_pos):
        for i in range(no_sequences):
            for j in range(no_wavelengths):
                mean_transducer_delta[i,j,k] = (mean_transducer[i,j,k] 
                                                - mean_transducer[i,(j-1)%no_wavelengths,k])        

        roll_wavelength_seq = np.argmax(np.mean(mean_transducer_delta[:,:,k], axis=(0)))
        print("start index for wavelength 680", roll_wavelength_seq)

        # reorder wavelengths by rolling
        mean_transducer[:,:,k] = np.roll(mean_transducer[:,:,k], -roll_wavelength_seq, axis = 1)
        mean_transducer_delta[:,:,k] = np.roll(mean_transducer_delta[:,:,k], -roll_wavelength_seq, axis = 1)
        std_transducer_left_m_right[:,:,k] = np.roll(std_transducer_left_m_right[:,:,k], -roll_wavelength_seq, axis = 1)
        raw_pa_sorted[:,:,:,:,k] = np.roll(raw_pa_sorted[:,:,:,:,k], -roll_wavelength_seq, axis = 3)

        roll_wavelength_seq = np.argmax(np.mean(mean_transducer_delta[:,:,k], axis=(0)))
        print("start index for wavelength 680 after rolling", roll_wavelength_seq)

    roll_illumination_seq = np.argmax(np.mean(std_transducer_left_m_right, axis=(0,1)))
    print("start index for leftmost illumination", roll_illumination_seq)

    # reorder illumination by rolling
    mean_transducer = np.roll(mean_transducer, -roll_illumination_seq, axis = 2)
    mean_transducer_delta = np.roll(mean_transducer_delta, -roll_illumination_seq, axis = 2)
    std_transducer_left_m_right = np.roll(std_transducer_left_m_right, -roll_illumination_seq, axis = 2)
    raw_pa_sorted = np.roll(raw_pa_sorted, -roll_illumination_seq, axis = 4)

    roll_illumination_seq = np.argmax(np.mean(std_transducer_left_m_right, axis=(0,1)))
    print("start index for leftmost illumination after rolling", roll_illumination_seq)
    # visual check for issues / framedrops
    # TODO autodetect frame drops 

    std_transducer_left_m_right = np.std(raw_pa_sorted[0:5, 20:40, ...], axis=(1,0)) - np.std(raw_pa_sorted[59:64, 20:40, ...], axis=(1,0))

    colors = ["r","g","b","y"]
    for k in range(no_ill_pos):
        for i in range(no_sequences):
            plt.plot(std_transducer_left_m_right[i,:,k], color=colors[k], alpha=0.3)
            
    plt.show()

    # convert to and save MITK compatible nrrd
    raw_pa_sorted = raw_pa_sorted.transpose(0,1,2,4,3).reshape(raw_pa_sorted.shape[0],raw_pa_sorted.shape[1],
                                                               raw_pa_sorted.shape[2]*raw_pa_sorted.shape[3]*raw_pa_sorted.shape[4])
    header = {'kinds': ['domain', 'domain', 'domain'], 
              'space': 'left-posterior-superior',
              'space directions': np.array([[0.3, 0, 0], [0, 1/20, 0], [0, 0, 0.1]]),
              'encoding': 'raw'
             }
    nrrd.write(acqusition+'/rawdata_pa.nrrd', raw_pa_sorted, header, index_order='F')
    print("[done]")

In [ ]:
## generate beamforming batch script ... this can then be run at your leisure
file_object = open(drive_path + 'process_all.sh', 'a')
for acqusition in acqusitions:
    command = (mitk_bin_path 
               + " -i \'" + acqusition + "/rawdata_pa.nrrd\'"
               + " -o \'" + acqusition + "/beamform_data_pa.nrrd\'"
               + " -s \'" + os.getcwd() + "/beamforming.options.xml" +"\'"
               + " -t PA")
    file_object.write(command)
    file_object.write("\n")
    
    command = (mitk_bin_path
               + " -i \'" + acqusition + "/beamform_data_pa.nrrd\'"
               + " -o \'" + acqusition + "/BMode_data_pa.nrrd\'"
               + " -s \'" + os.getcwd() + "/bandpass.bmode.options.xml" +"\'"
               + " -t PA")
    file_object.write(command)
    file_object.write("\n")
    command = "rm " + acqusition + "/beamform_data_pa.nrrd"
    file_object.write(command)
    file_object.write("\n")
file_object.close()